<a href="https://colab.research.google.com/github/laplezeda/agents/blob/main/Copy_of_Final_CrewAI_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Outreach Campaign

We learn how to create custom tools for a customer outreach campaign. A Good tool has 3 key elements:

- Versatility
- Caching
- Fault Tolerance

Note: CrewAI only works with Python version >=3.10 and <=3.13

In [ ]:
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


## Step 01. Installing CrewAI packages

In [ ]:
%%capture
!pip install crewai

In [ ]:
%%capture
!pip install 'crewai[tools]'

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing CrewAI library (Agent, Task and Crew module)

In [ ]:
from crewai import Agent, Task, Crew

To use Agents, we need to set env. variables (key, model name)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ['SERPER_API_KEY'] = ''

## Step 03. Creating Agents

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Step 04. Creating Tools

### CrewAI Tools

- **SerperDevTool** : Used for Google Search
- **DirectoryReadTool** : Open a directory
- **FileReadTool** : Read a file

In [ ]:
from crewai_tools import SerperDevTool, DirectoryReadTool, FileReadTool

In [ ]:
# local directory 'instructions' contains markdown files
# markdown files (.md) having instructions on how to deal with company of different sizes

directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
serper_dev_tool = SerperDevTool()

### Custom Tool (SentimentAnalysisTool)

- Create a custom tool using crewAI's BaseTool class

In [ ]:
from crewai.tools import BaseTool


- Every Tool needs to have a `name` and a `description`.
- Tool logic written in `_run` function
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name : str = "Sentiment Analysis Tool"
    description : str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, argument : str) -> str:
        return "positive"


In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()


## Step 05. Creating Tasks

- Lead Profiling Task usign crewAI Tools

In [ ]:
lead_profiling = Task (
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[serper_dev_tool, directory_read_tool, file_read_tool],
    agent=sales_rep_agent
)

- The personalized outreach task is created using custom tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search tool)

In [ ]:
personalized_outreach_task = Task (
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"

    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[serper_dev_tool, sentiment_analysis_tool],
    agent=lead_sales_rep_agent
)

## Step 06.  Creating a crew

In [ ]:
crew = Crew(
    tasks=[lead_profiling, personalized_outreach_task],
    agents=[sales_rep_agent, lead_sales_rep_agent],
    memory=True,
    verbose=False
)

## Kicking off the crew

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that    │
│  recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile,   │
│  focusing on key decision-makers, recent business developments, and potential needs that align with our         │
│  offerings. This task is crucial for tailoring our engagement strategy effectively.                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: Thought: To start the analysis on DeepLearningAI, I need to gather comprehensive data about the       │
│  company. This will include background information, key personnel, recent business activities, and potential    │
│  areas where our solutions might be of value. I will first search the internet to find the latest news and      │
│  general information about DeepLearningAI.                                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"DeepLearningAI company profile\"}"                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'DeepLearningAI company profile', 'type': 'search', 'num': 10, 'engine':            │
│  'google'}, 'organic': [{'title': 'About - DeepLearning.AI', 'link': 'https://www.deeplearning.ai/about/',      │
│  'snippet': 'DeepLearning.AI is an education technology company that is empowering the global workforce to      │
│  build an AI-powered future through world-class education, hands-on training, and a collaborative community     │
│  .', 'position': 1}, {'title': 'DeepLearning.AI - LinkedIn', 'link':                                            │
│  'https://www.linkedin.com/company/deeplearningai', 'snippet': 'DeepLearning.AI is making a world-class AI      │
│  education accessible to people around the globe. DeepLearning.AI was founded by Andrew Ng, a global leader in  │
│  AI.', 'position': 2}, {'title': 'Deeplearning.ai - Crunchbase Company Profile & Funding', 'link':              │
│  'https://www.crunchbase.com/organization/deeplearning-ai', 'snippet': 'Deeplearning.ai is company that         │
│  explores the frontier of AI. The company is founded by Andrew Ng, the former Chief Scientist of Baidu and the  │
│  current ...', 'position': 3}, {'title': 'DeepLearning.AI: Start or Advance Your Career in AI', 'link':         │
│  'https://www.deeplearning.ai/', 'snippet': 'Join over 7 million people learning how to use and build AI. AI    │
│  Courses and Specializations. Build a foundation of machine learning and AI skills.', 'position': 4,            │
│  'sitelinks': [{'title': 'About', 'link': 'https://www.deeplearning.ai/about/'}, {'title': 'Careers', 'link':   │
│  'https://www.deeplearning.ai/careers/'}, {'title': 'Courses', 'link':                                          │
│  'https://www.deeplearning.ai/courses/'}, {'title': 'DLAI Logo', 'link':                                        │
│  'https://learn.deeplearning.ai/login'}]}, {'title': 'Deeplearning - 2025 Company Profile, Funding &            │
│  Competitors', 'link':                                                                                          │
│  'https://tracxn.com/d/companies/deeplearning/__SekrcD6K_o4Cvm4SgstSYBIp1zHk-3FzPsR3SkeAgnE', 'snippet':        │
│  'Deeplearning is a funded company based in Palo Alto (United States), founded in 2017 by Andrew Ng. It         │
│  operates as an Online platform ...', 'position': 5}, {'title': 'DEEPLEARNING.AI LLC Company ...                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: Thought: The search results have provided a solid foundation for the company profile of               │
│  DeepLearningAI. It's important to document key details such as background information, key personnel, and      │
│  recent milestones. Andrew Ng's role as a founder is crucial, and I need to explore more specific details       │
│  about recent business activities and customers' needs.                                                         │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Thought: To create effective personalized outreach emails for Andrew Ng, CEO of DeepLearningAI, I need to      │
│  connect our solutions directly to their recent product launch ("AI for Everyone") and their goals of           │
│  expanding AI education globally. It's important to align the tone of the emails with the professional,         │
│  forward-thinking spirit of DeepLearningAI and to clearly showcase how our tools can aid their rapid            │
│  development of AI technologies, particularly in practical applications.                                        │
│  I will use the Sentiment Analysis Tool to ensure that the communications maintain a positive tone before       │
│  finalizing the drafts of the emails.                                                                           │
│  Thought: I need to create engaging and relevant email drafts for Andrew Ng.                                    │
│                                                                                                                 │
│  Using Tool: Sentiment Analysis Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"argument\": \"Dear Andrew Ng, I hope this email finds you well. \\nI am reaching out to congratulate you   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  positive                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Email 1 - Educational Collaboration Focus**                                                               │
│     ```                                                                                                         │
│     Subject: Empowering the Next Wave of AI Innovators Together                                                 │
│                                                                                                                 │
│     Dear Andrew,                                                                                                │
│                                                                                                                 │
│     I was immensely impressed by DeepLearningAI's recent launch of 'AI for Everyone.' At CrewAI, we believe in  │
│  the power of education to transform industries and lives. Given our shared commitment to making AI             │
│  accessible, I see a significant opportunity for synergy between our teams.                                     │
│                                                                                                                 │
│     We propose to co-develop a specialized course that delves into the practical applications of AI in          │
│  industries like healthcare and automotive, leveraging both our advanced AI tools and your educational          │
│  framework. This could serve as a cornerstone for not only educating but truly empowering professionals to      │
│  implement AI in real-world scenarios.                                                                          │
│                                                                                                                 │
│     Let's set up a time to discuss how we can integrate our resources for maximal impact.                       │
│                                                                                                                 │
│     Best regards,                                                                                               │
│     [Your Name]                                                                                                 │
│     Lead Sales Representative, CrewAI                                                                           │
│     ```                                                                                                         │
│                                                                                                                 │
│  2. **Email 2 - Technology Integration Focus**                                                                  │
│     ```                                                                                                         │
│     Subject: Enhancing DeepLearningAI's Platform through Advanced AI Tools                                      │
│                                                                                                                 │
│     Dear Andrew,                                                                                                │
│                                                                                                                 │
│     Congratulations on your latest successes and the impactful launch of AI courses aimed at business           │
│  professionals. CrewAI's platform features advanced AI 

/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [ ]:
from IPython.display import Markdown
import re

content = result.raw
markdown_content = re.sub(r'```(?:markdown)?\n?(.*?)\n?```', r'\1', content, flags=re.DOTALL).strip()
Markdown(markdown_content)

1. **Email 1 - Educational Collaboration Focus**
      Subject: Empowering the Next Wave of AI Innovators Together

   Dear Andrew,

   I was immensely impressed by DeepLearningAI's recent launch of 'AI for Everyone.' At CrewAI, we believe in the power of education to transform industries and lives. Given our shared commitment to making AI accessible, I see a significant opportunity for synergy between our teams.

   We propose to co-develop a specialized course that delves into the practical applications of AI in industries like healthcare and automotive, leveraging both our advanced AI tools and your educational framework. This could serve as a cornerstone for not only educating but truly empowering professionals to implement AI in real-world scenarios.

   Let's set up a time to discuss how we can integrate our resources for maximal impact.

   Best regards,
   [Your Name]
   Lead Sales Representative, CrewAI
   

2. **Email 2 - Technology Integration Focus**
      Subject: Enhancing DeepLearningAI's Platform through Advanced AI Tools

   Dear Andrew,

   Congratulations on your latest successes and the impactful launch of AI courses aimed at business professionals. CrewAI's platform features advanced AI tools that could enhance these educational offerings. Our tools are designed for rapid prototyping and deployment, which are ideal for hands-on learning environments.

   By incorporating our tools directly into DeepLearningAI’s courses, we can provide an enriched, practical learning experience that directly translates into skills that can be applied in the market today.

   I believe a short meeting could be very productive for us to explore potential integrations and discuss the specific needs of your platform.

   Looking forward to your thoughts,

   Warm regards,
   [Your Name]
   Lead Sales Representative, CrewAI
   
   
3. **Email 3 - Partnership in Research and Development Focus**
      Subject: Collaborative R&D Opportunities in AI

   Dear Andrew,

   DeepLearningAI's recent initiatives and funding achievements inspire a vision for what's possible in AI research and development. CrewAI is at the forefront of developing practical AI applications, and we see immense potential in partnering with an innovator like yourself.

   We envision a joint R&D program where our combined expertise leads to breakthroughs in AI application across various sectors. This initiative could not only advance the field but also provide real-world case studies for educational purposes.

   Could we schedule a time next week to explore this potential partnership further?

   Sincerely,
   [Your Name]
   Lead Sales Representative, CrewAI
   

These email drafts are tailored to engage Andrew Ng directly, addressing the strategic interests of DeepLearningAI while highlighting how CrewAI’s solutions align with and can enhance their goals.